In [ ]:
import pandas as pd
import numpy  as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### function to convert the time in the specific format

In [ ]:
def ampm(x):
    dat = x.split()
    if(dat[1]=='PM'):
        temp1 = dat[0].split(":")
        temp = str(int(temp1[0])+12)
        if(temp=='24'):
            temp = "00"
        fin = temp +":"+temp1[1]
    else:
        fin = dat[0]
    return fin

In [ ]:
def preprocess(year):
    for i in range(1,13):
    if(i<10):
        month = "0"+str(i)
    else:
        month = str(i)
    #print(month)
    if((i==1) | (i==3) | (i==5) | (i==7) | (i==8) | (i==10) | (i==12)):
        for j in range(1,32):
            if(j<10):
                date = "0"+str(j)
            else:
                date = str(j)
            try:
                df= pd.read_csv(string1+"-"+month+"-"+date+'.csv')
                df['time'] = df['Time'].map(ampm)
                df['time'] = df['time']+ " " + df['Date']
                df['time'] = pd.to_datetime(df['time'],format = "%H:%M %Y-%m-%d")
                df.to_csv(string2+"-"+month+"-"+date+'.csv')
            except:
                print(date,month)
    elif(i==2):
        for j in range(1,28):
            if(j<10):
                date = "0"+str(j)
            else:
                date = str(j)
            try:
                df= pd.read_csv(string1+"-"+month+"-"+date+'.csv')
                df['time'] = df['Time'].map(ampm)
                df['time'] = df['time']+ " " + df['Date']
                df['time'] = pd.to_datetime(df['time'],format = "%H:%M %Y-%m-%d")
                df.to_csv(string2+"-"+month+"-"+date+'.csv')
            except:
                print(date,month)
    else:
        for j in range(1,31):
            if(j<10):
                date = "0"+str(j)
            else:
                date = str(j)
            try:
                df= pd.read_csv(string1+"-"+month+"-"+date+'.csv')
                df['time'] = df['Time'].map(ampm)
                df['time'] = df['time']+ " " + df['Date']
                df['time'] = pd.to_datetime(df['time'],format = "%H:%M %Y-%m-%d")
                df.to_csv(string2+"-"+month+"-"+date+'.csv')
            except:
                print(date,month)
    return 1

In [ ]:
years = [2016,2017,2018]
for i in years:
    string1 = "./"+str(i)+".csv"
    string2 = "./output/"+str(i)+"csv"
    month,date="",""
    preprocess(i)

In [ ]:
def merge(year):
    string1 = './'+year+".csv"
    month,date="",""
    cnt=0
    for i in range(1,13):
        if(i<10):
            month = "0"+str(i)
        else:
            month = str(i)
        #print(month)
        if((i==1) | (i==3) | (i==5) | (i==7) | (i==8) | (i==10) | (i==12)):
            for j in range(1,32):
                if(j<10):
                    date = "0"+str(j)
                else:
                    date = str(j)
                df= pd.read_csv(string1+"-"+month+"-"+date+'.csv')
                df.drop(['Time','Date'],axis=1,inplace=True)
                cnt=cnt+1
                if((i==1) & (cnt==1)):
                    df1= df
                else:
                    df1= pd.concat([df1,df])
                #print(date)
        elif(i==2):
            for j in range(1,28):
                if(j<10):
                    date = "0"+str(j)
                else:
                    date = str(j)
                df= pd.read_csv(string1+"-"+month+"-"+date+'.csv')
                df.drop(['Time','Date'],axis=1,inplace=True)
                df1= pd.concat([df1,df])
        else:
            for j in range(1,31):
                if(j<10):
                    date = "0"+str(j)
                else:
                    date = str(j)
                df= pd.read_csv(string1+"-"+month+"-"+date+'.csv')
                df.drop(['Time','Date'],axis=1,inplace=True)
                df1= pd.concat([df1,df])
    df1.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace=True)
    df1.reset_index(inplace=True)
    df1.drop('index',axis=1,inplace=True)
    df1.to_csv('./merge'+year+'.csv')
    return 1

In [ ]:
years = [2016,2017,2018]
for i in years:
    merge(i)

In [ ]:
flights = pd.read_csv("./flights.csv")

In [ ]:
atl = flights[flights['DEST']=='ATL'].copy(deep=True)

In [ ]:
atl.drop(['Unnamed: 27','OP_CARRIER','OP_CARRIER_FL_NUM','SECURITY_DELAY','NAS_DELAY','CANCELLATION_CODE','TAXI_OUT','ARR_TIME'],axis=1,inplace=True)
atl.head()

In [ ]:
weat = pd.read_csv('./merge.csv',index_col='Unnamed: 0')
weat.head()

In [ ]:
names = weat.drop('time',axis=1).columns
atl[names] = 0
atl.head()

In [ ]:
import missingno as ms
ms.matrix(atl)

In [ ]:
atl = atl[atl['DEP_TIME'].isna()==False]

In [ ]:
atl.loc[atl['DEP_TIME']>=2400,'DEP_TIME'] = 0

In [ ]:
atl['time'] = atl['DEP_TIME'].astype('str').str.split(".",expand=True)[0]

In [ ]:
def timest(x):
    if(len(x)==2):
        x= "00"+x
    elif(len(x)==1):
        x="000"+x
    return x
atl['time']=atl['time'].map(timest)

In [ ]:
weat['time'] = pd.to_datetime(weat['time'],format="%Y-%m-%d %H:%M:%S")

In [ ]:
atl['time'] = atl['time'] + " "+ atl['FL_DATE']

## the final loop for value imputations

In [ ]:
arr = []
for i in range(1,13):
    if((i==1) | (i==3) | (i==5) | (i==7) | (i==8) | (i==10) | (i==12)):
        for j in range(1,32):
            for k in weat[(weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['time'].dt.hour.unique():
                arr.append(np.array(weat[(weat['time'].dt.day==j) & (weat['time'].dt.month==i) & (weat['time'].dt.hour==k)])[0])
    elif(i==2):
        for j in range(1,28):
            for k in weat[(weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['time'].dt.hour.unique():
                arr.append(np.array(weat[(weat['time'].dt.day==j) & (weat['time'].dt.month==i) & (weat['time'].dt.hour==k)])[0])
    else:
        for j in range(1,31):
            for k in weat[(weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['time'].dt.hour.unique():
                arr.append(np.array(weat[(weat['time'].dt.day==j) & (weat['time'].dt.month==i) & (weat['time'].dt.hour==k)])[0])


In [ ]:
atl['time'] = pd.to_datetime(atl['time'],format="%H%M %Y-%m-%d")

In [ ]:
temp = {}
for i in range(0,len(arr)):
    temp[i] = arr[i]
weather = pd.DataFrame(temp).transpose()
weather.columns = weat.columns

In [ ]:
for i in range(1,13):
    for j in range(1,32):
        hoursunique = ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i)]['time'].dt.hour.unique()
        for k in hoursunique:
            if(len(weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Temperature'].values)>0):
                ans = weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Temperature'].values[0]
                #print(ans)
                ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Temperature'] = ans
                print(ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Temperature'])

for i in range(1,13):
    for j in range(1,32):
        hoursunique = ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i)]['time'].dt.hour.unique()
        for k in hoursunique:
            if(len(weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Condition'].values)>0):
                ans = weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Condition'].values[0]
                #print(ans)
                ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Condition'] = ans
                print(ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Condition'])

for i in range(1,13):
    for j in range(1,32):
        hoursunique = ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i)]['time'].dt.hour.unique()
        for k in hoursunique:
            if(len(weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Wind Speed'].values)>0):
                ans = weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Wind Speed'].values[0]
                #print(ans)
                ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Wind Speed'] = ans
                print(ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Wind Speed'])

for i in range(1,13):
    for j in range(1,32):
        hoursunique = ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i)]['time'].dt.hour.unique()
        for k in hoursunique:
            if(len(weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Dew Point'].values)>0):
                ans = weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Dew Point'].values[0]
                #print(ans)
                ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Dew Point'] = ans
                print(ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Dew Point'])

for i in range(1,13):
    for j in range(1,32):
        hoursunique = ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i)]['time'].dt.hour.unique()
        for k in hoursunique:
            if(len(weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Humidity'].values)>0):
                ans = weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Humidity'].values[0]
                #print(ans)
                ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Humidity'] = ans
                print(ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Humidity'])

for i in range(1,13):
    for j in range(1,32):
        hoursunique = ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i)]['time'].dt.hour.unique()
        for k in hoursunique:
            if(len(weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Pressure'].values)>0):
                ans = weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Pressure'].values[0]
                #print(ans)
                ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Pressure'] = ans
                print(ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Pressure'])

for i in range(1,13):
    for j in range(1,32):
        hoursunique = ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i)]['time'].dt.hour.unique()
        for k in hoursunique:
            if(len(weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Precipitation'].values)>0):
                ans = weat[(weat['time'].dt.hour==k) & (weat['time'].dt.day==j) & (weat['time'].dt.month==i)]['Precipitation'].values[0]
                #print(ans)
                ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Precipitation'] = ans
                print(ATL.loc[(ATL['time'].dt.day==j) & (ATL['time'].dt.month==i) & (ATL['time'].dt.hour==k) ,'Precipitation'])